<a href="https://colab.research.google.com/github/desve/vk-text-to-image-demo/blob/main/04_service_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Установка и импорт

In [ ]:
!pip install -q diffusers transformers accelerate safetensors

import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image
import io, os

# 2. Инициализация модели

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

pipe.enable_attention_slicing()

# 3. Базовые стили и промпты

In [ ]:
main_styles = ["vk_flat", "neon"]

base_prompts = {
    "Самопрезентация": (
        "one young AI engineer on stage with microphone, "
        "big screen behind showing neural network diagram, tech conference"
    ),
    "Профессиональное достижение": (
        "data scientist working late at night in a modern office, "
        "only monitor light and city lights outside window, charts and neural networks on screens"
    ),
    "Команда VK": (
        "three interns in a modern IT office, sitting around a round table with laptops and sticky notes, "
        "warm friendly atmosphere"
    )
}

def build_prompt(base, style_key):
    return f"{base}, {styles[style_key]}"

In [ ]:
# Генерация уменьшенного изображения
def generate_one(
    prompt,
    steps=25,
    guidance=7.5,
    seed=None,
    height=512,
    width=512
):
    generator = None
    if seed is not None:
        generator = torch.Generator(device=device).manual_seed(seed)

    image = pipe(
        prompt,
        num_inference_steps=steps,
        guidance_scale=guidance,
        num_images_per_prompt=1,
        generator=generator,
        height=height,
        width=width
    ).images[0]
    return image

In [ ]:
for scene_name, base in base_prompts.items():
    print("\n=== ", scene_name, " ===")
    for style_key in main_styles:
        full_prompt = build_prompt(base, style_key)
        print("Стиль:", style_key)
        img = generate_one(full_prompt, steps=25, guidance=7.5,
                           height=512, width=512)
        display(img)
        filename = f"{scene_name}_{style_key}.jpg".replace(" ", "_")
        save_compressed(img, filename, quality=75)